### Imports

In [1]:
import sys
sys.dont_write_bytecode = True


import torch
import numpy as np
import random
import os

def set_seeds(seed_value=42):
    """Sets seeds for reproducibility."""
    random.seed(seed_value)
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed_value)

set_seeds(42) 

import json
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import warnings
import logging
from datetime import datetime
warnings.filterwarnings('ignore')

from model import get_model
from config import CFG
from dataset import OfficeHomeDataset
from plot import plot_domainwise_accuracy
from transform import get_transforms
from runner import run_baseline, run_lodo

torch.manual_seed(CFG["system"]["seed"])
np.random.seed(CFG["system"]["seed"])

device = CFG["system"]["device"]
print(f"Device: {device}")
print(f"PyTorch: {torch.__version__}")


Device: cuda
PyTorch: 2.5.1+cu121


### DataLoading

In [2]:
train_transform, test_transform = get_transforms(img_size=224, augment=False, use_imagenet_norm=False)

oh = OfficeHomeDataset(
    data_root=CFG["datasets"]["OfficeHome"]["root"],
    transform=train_transform,
    batch_size=CFG["train"]["batch_size"]
)

print("\nData loaders ready!")


Data loaders ready!


### Logging

In [3]:

dataset_name = "OfficeHome"
base_dir = os.path.join(os.getcwd(), dataset_name)
subdirs = ["logs", "checkpoints", "plots"]

for sub in subdirs:
    os.makedirs(os.path.join(base_dir, sub), exist_ok=True)

timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
log_file = os.path.join(base_dir, "logs", f"train_{timestamp}.log")

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s | %(levelname)s | %(message)s",
    handlers=[logging.FileHandler(log_file)]
)

logger = logging.getLogger(dataset_name)

logger.info(f"Initialized experiment directories for {dataset_name}")
logger.info(f"Logs: {os.path.join(base_dir, 'logs')}")
logger.info(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
logger.info(f"Plots: {os.path.join(base_dir, 'plots')}")

print(f"Initialized experiment directories for {dataset_name}")
print(f"Logs: {os.path.join(base_dir, 'logs')}")
print(f"Checkpoints: {os.path.join(base_dir, 'checkpoints')}")
print(f"Plots: {os.path.join(base_dir, 'plots')}")


Initialized experiment directories for OfficeHome
Logs: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\OfficeHome\logs
Checkpoints: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\OfficeHome\checkpoints
Plots: c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\OfficeHome\plots


### Setup

In [4]:
domains = CFG["datasets"]["OfficeHome"]["domains"]
loaders = {d: {"train": oh.get_dataloader(d, train=True), "val": oh.get_dataloader(d, train=False)} for d in domains}
ckpt_root = os.path.join(base_dir, "checkpoints")
log_dir = os.path.join(base_dir, "logs")
plots_dir = os.path.join(base_dir, "plots")
os.makedirs(ckpt_root, exist_ok=True)
os.makedirs(log_dir, exist_ok=True)
os.makedirs(plots_dir, exist_ok=True)
model_factory = lambda cfg, dataset_key: get_model(cfg,dataset="OfficeHome")
optimizer_fn = lambda model: optim.AdamW(model.parameters(), lr=CFG["train"]["lr"], weight_decay=CFG["train"].get("weight_decay", 0.01))
device = CFG["system"]["device"]
epochs = CFG["train"]["epochs"]


{
  "lodo_results": {
    "art_painting": 0.8341463414634146,
    "cartoon": 0.7974413646055437,
    "photo": 0.9580838323353293,
    "sketch": 0.6017811704834606
  },
  "timestamp": "20251004_020611"
}

### Leave One Domain Out

In [5]:
lodo_results, lodo_mean, lodo_summary = run_lodo(
    model_fn=model_factory,
    CFG=CFG,
    logger=logger,
    dataset_key="OfficeHome",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    ckpt_root=ckpt_root,
    log_dir=log_dir,
    epochs=epochs
)

Some weights of ViTModel were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== LODO: Leaving out domain 'Art' ===


Evaluating: 100%|██████████| 19/19 [00:13<00:00,  1.46it/s]


[Art] Epoch 1/5 | Train - Loss: 2.9347, Cls: 2.9256, GRQO: 0.0091, Acc: 0.3950 | Val - Loss: 2.1056, Cls: 2.1035, GRQO: 0.0022, Acc: 0.5550
[Art] New best val acc: 0.5550


Evaluating: 100%|██████████| 19/19 [00:11<00:00,  1.71it/s]


...

[Real World] Epoch 3/5 | Train - Loss: 0.6431, Cls: 0.6393, GRQO: 0.0037, Acc: 0.8857 | Val - Loss: 0.8849, Cls: 0.8842, GRQO: 0.0007, Acc: 0.7838
[Real World] New best val acc: 0.7838


Evaluating: 100%|██████████| 35/35 [00:21<00:00,  1.64it/s]


[Real World] Epoch 4/5 | Train - Loss: 0.3198, Cls: 0.3165, GRQO: 0.0033, Acc: 0.9509 | Val - Loss: 0.7930, Cls: 0.7925, GRQO: 0.0005, Acc: 0.8006
[Real World] New best val acc: 0.8006


Evaluating: 100%|██████████| 35/35 [00:21<00:00,  1.65it/s]

[Real World] Epoch 5/5 | Train - Loss: 0.1719, Cls: 0.1690, GRQO: 0.0029, Acc: 0.9777 | Val - Loss: 0.7734, Cls: 0.7730, GRQO: 0.0005, Acc: 0.8033
[Real World] New best val acc: 0.8033
[Real World] Best Acc: 0.8033
------------------------------------------------------------
LODO finished | Mean Acc: 0.6950
Summary saved to c:\Users\Fatim_Sproj\Desktop\Fatim\Spring 2025\sproj\Visual-Reasoning\Vit-GRQO\vit-tiny\OfficeHome\logs\lodo_summary_20251010_193546.json


### Baseline

In [6]:
model_name = "WinKawaks/vit-tiny-patch16-224"
baseline_results, baseline_mean = run_baseline(
    model_name=model_name,
    CFG=CFG,
    logger=logger,
    dataset_key="OfficeHome",
    domains=domains,
    loaders=loaders,
    optimizer_fn=optimizer_fn,
    device=device,
    epochs=CFG["train"]["epochs"]
)


Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([65]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([65, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'Art' ===
[Art] Epoch 1/5 | Train - Loss: 1.7685, Acc: 0.5955 | Val Acc: 0.5834
[Art] Epoch 2/5 | Train - Loss: 0.4521, Acc: 0.8946 | Val Acc: 0.6246
[Art] Epoch 3/5 | Train - Loss: 0.1791, Acc: 0.9647 | Val Acc: 0.6300
[Art] Epoch 4/5 | Train - Loss: 0.0865, Acc: 0.9831 | Val Acc: 0.6300
[Art] Epoch 5/5 | Train - Loss: 0.0555, Acc: 0.9872 | Val Acc: 0.6428
[Art] Best Val Acc: 0.6428
------------------------------------------------------------
Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([65]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([65, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'Clipart' ===
[Clipart] Epoch 1/5 | Train - Loss: 1.8172, Acc: 0.5814 | Val Acc: 0.4696
[Clipart] Epoch 2/5 | Train - Loss: 0.4404, Acc: 0.9009 | Val Acc: 0.5129
[Clipart] Epoch 3/5 | Train - Loss: 0.1565, Acc: 0.9734 | Val Acc: 0.5372
[Clipart] Epoch 4/5 | Train - Loss: 0.0638, Acc: 0.9909 | Val Acc: 0.5276
[Clipart] Epoch 5/5 | Train - Loss: 0.0357, Acc: 0.9951 | Val Acc: 0.5375
[Clipart] Best Val Acc: 0.5375
------------------------------------------------------------
Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([65]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([65, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'Product' ===
[Product] Epoch 1/5 | Train - Loss: 2.0915, Acc: 0.5183 | Val Acc: 0.7189
[Product] Epoch 2/5 | Train - Loss: 0.6015, Acc: 0.8565 | Val Acc: 0.7605
[Product] Epoch 3/5 | Train - Loss: 0.2429, Acc: 0.9517 | Val Acc: 0.7572
[Product] Epoch 4/5 | Train - Loss: 0.1285, Acc: 0.9775 | Val Acc: 0.7725
[Product] Epoch 5/5 | Train - Loss: 0.0724, Acc: 0.9850 | Val Acc: 0.7740
[Product] Best Val Acc: 0.7740
------------------------------------------------------------
Initializing ViT baseline: WinKawaks/vit-tiny-patch16-224


Some weights of ViTForImageClassification were not initialized from the model checkpoint at WinKawaks/vit-tiny-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([65]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 192]) in the checkpoint and torch.Size([65, 192]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Baseline LODO: Leaving out domain 'Real World' ===
[Real World] Epoch 1/5 | Train - Loss: 2.0106, Acc: 0.5341 | Val Acc: 0.7448
[Real World] Epoch 2/5 | Train - Loss: 0.5484, Acc: 0.8729 | Val Acc: 0.7822
[Real World] Epoch 3/5 | Train - Loss: 0.2108, Acc: 0.9586 | Val Acc: 0.7994
[Real World] Epoch 4/5 | Train - Loss: 0.0985, Acc: 0.9825 | Val Acc: 0.7957
[Real World] Epoch 5/5 | Train - Loss: 0.0607, Acc: 0.9876 | Val Acc: 0.8033
[Real World] Best Val Acc: 0.8033
------------------------------------------------------------
Baseline LODO (WinKawaks/vit-tiny-patch16-224) finished | Mean Acc: 0.6894
